In [1]:
import cv2
import numpy as np
from datetime import datetime
from collections import deque

# ===== 作業抬頭設定 =====
STUDENT_ID = "學號: 611406012"   # ← 已替你填好
OUT_NAME   = "output_green.mp4"  # 影片檔名
TARGET_FPS = 20.0                # 錄影 FPS

# 建議的綠色 HSV 範圍（視現場光線可微調）
LOWER_GREEN = np.array([35, 70, 70])    # H:35~85 大致是綠色
UPPER_GREEN = np.array([85, 255, 255])

def put_texts(img, fps):
    now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    cv2.putText(img, now, (10, 24), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,255,255), 2, cv2.LINE_AA)
    cv2.putText(img, f"FPS:{fps:.1f}", (10, 52), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,255,255), 2, cv2.LINE_AA)
    cv2.putText(img, STUDENT_ID, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255,255,0), 2, cv2.LINE_AA)
    cv2.putText(img, "Color Range: Green (HSV 35~85)", (10, img.shape[0]-12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ 無法開啟攝影機")
        return

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUT_NAME, fourcc, TARGET_FPS, (w, h))

    cv2.namedWindow("mask", cv2.WINDOW_NORMAL)
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("mask", 480, 360)
    cv2.resizeWindow("output", 640, 360)

    # 平滑 FPS
    times = deque(maxlen=30)
    t_prev = cv2.getTickCount()

    print("📸 偵測綠色物體中… 按 q 結束。")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ 無法讀取畫面")
            break

        # FPS
        t_now = cv2.getTickCount()
        dt = (t_now - t_prev) / cv2.getTickFrequency()
        t_prev = t_now
        times.append(dt)
        fps = 1.0 / (sum(times)/len(times)) if times else TARGET_FPS

        # HSV + inRange 取綠色
        hsv  = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, LOWER_GREEN, UPPER_GREEN)

        # 去雜訊
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  k, iterations=1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=1)

        # 套遮罩
        output = cv2.bitwise_and(frame, frame, mask=mask)

        # 浮水印/資訊
        put_texts(output, fps)

        # 顯示 + 錄影
        cv2.imshow("mask", mask)
        cv2.imshow("output", output)
        out.write(output)

        if (cv2.waitKey(1) & 0xFF) == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"✅ 已輸出：{OUT_NAME}")

if __name__ == "__main__":
    main()


📸 偵測綠色物體中… 按 q 結束。
✅ 已輸出：output_green.mp4
